In [ ]:
# 0. Библиотеки и настройки
import gc
import random
import zipfile

import numpy as np
import pandas as pd
import torch
import evaluate
from seqeval.metrics import classification_report
from sklearn.metrics import confusion_matrix
from tqdm.notebook import tqdm
from IPython.display import display, HTML

from datasets import Dataset, concatenate_datasets
from corus import load_ne5
import corus.sources.ne5 as ne5
import razdel
from razdel.substring import Substring
from dataclasses import dataclass

from transformers import (
    AutoModelForMaskedLM,
    AutoModelForTokenClassification,
    AutoTokenizer,
    DataCollatorForLanguageModeling,
    DataCollatorForTokenClassification,
    Trainer,
    TrainingArguments,
    pipeline,
)


random_state = 42

# 1. Preprocessing

In [2]:
# !curl http://www.labinform.ru/pub/named_entities/collection5.zip -L -o data/collection5.zip
# 
# with zipfile.ZipFile("data/collection5.zip", 'r') as z:
#     z.extractall("data/")

Загрузка и парсинг аннотированного корпуса NE5

In [3]:
def load_text_utf8(path):
    with open(path, 'r', encoding='utf-8') as f:
        return f.read()

ne5.load_text = load_text_utf8
corpus = list(load_ne5("data/Collection5/"))

In [4]:
@dataclass
class Ne5Span:
    start: int
    stop: int
    type: str

@dataclass
class LabeledText:
    text: str
    entities: list[Ne5Span]


In [5]:
def parse_ds_ner_fixed(corpus: list) -> tuple[list[LabeledText], dict[str, int]]:
    labeled_texts = []
    cats_map = {}
    for annot in corpus:
        entities = []
        for ent in annot.spans:
            new_ent = Ne5Span(
                start=ent.start,
                stop=ent.stop,
                type=ent.type
            )
            entities.append(new_ent)
            if new_ent.type not in cats_map:
                cats_map[new_ent.type] = len(cats_map)
        entities.sort(key=lambda x: x.start)
        lab_txt = LabeledText(
            text=annot.text,
            entities=entities
        )
        labeled_texts.append(lab_txt)
    return labeled_texts, cats_map


In [ ]:
# 1.1 Сплит train/test
def train_test_split(data: list[LabeledText], test_n: int, random_seed: int = 0) -> tuple[list[LabeledText], list[LabeledText]]:
    total_n = len(data)
    print(f'train data: {total_n - test_n}')
    print(f'test data: {test_n}')
    data_idxs = range(total_n)
    random.seed(random_seed)
    test_idxs = sorted(random.sample(data_idxs, test_n))
    train_idxs = sorted(set(data_idxs) - set(test_idxs))
    data_test = [data[test_img_idx] for test_img_idx in test_idxs]
    data_train = [data[train_img_idx] for train_img_idx in train_idxs]
    return data_train, data_test

In [7]:
labeled_text, categories_map = parse_ds_ner_fixed(corpus)

print(f'texts_n: {len(labeled_text)}')
print(categories_map)

texts_n: 1000
{'GEOPOLIT': 0, 'LOC': 1, 'MEDIA': 2, 'PER': 3, 'ORG': 4}


In [8]:
print(labeled_text[0])

LabeledText(text='Россия рассчитывает на конструктивное воздействие США на Грузию\n\n04/08/2008 12:08\n\nМОСКВА, 4 авг - РИА Новости. Россия рассчитывает, что США воздействуют на Тбилиси в связи с обострением ситуации в зоне грузино-осетинского конфликта. Об этом статс-секретарь - заместитель министра иностранных дел России Григорий Карасин заявил в телефонном разговоре с заместителем госсекретаря США Дэниэлом Фридом.\n\n"С российской стороны выражена глубокая озабоченность в связи с новым витком напряженности вокруг Южной Осетии, противозаконными действиями грузинской стороны по наращиванию своих вооруженных сил в регионе, бесконтрольным строительством фортификационных сооружений", - говорится в сообщении.\n\n"Россия уже призвала Тбилиси к ответственной линии и рассчитывает также на конструктивное воздействие со стороны Вашингтона", - сообщил МИД России. ', entities=[Ne5Span(start=0, stop=6, type='GEOPOLIT'), Ne5Span(start=50, stop=53, type='GEOPOLIT'), Ne5Span(start=57, stop=63, type

In [9]:
train_data, test_data = train_test_split(
    labeled_text,
    test_n=200,
    random_seed = random_state,
)

train data: 800
test data: 200


## Visualisation

In [10]:
COLOR_MAP = {
    'GEOPOLIT': 'yellow',
    'LOC': 'lightblue',
    'MEDIA': "lightgreen",
    'PER': "pink",
    'ORG': "lightgrey",
}

In [11]:
def visualize_labeled_text(labeled_text: LabeledText, color_map: dict[str, str]):
    """
    Visualize the labeled data for Named Entity Recognition (NER).

    Args:
        labeled_text: An instance of LabeledText containing the text and entities.
        color_map: A color map for each entity category.

    Returns:
        None. Displays the HTML representation of the text with highlighted entities.
    """
    # Initialize the HTML string
    labeled_text2 = labeled_text
    labeled_text2.text = labeled_text.text.replace("\n", "  ")
    html_str = ""
    last_index = 0
    # Sort entities by the starting index
    entities = sorted(labeled_text2.entities, key=lambda x: x.start)
    for entity in entities:
        start, stop, type = entity.start, entity.stop, entity.type
        html_str += labeled_text2.text[last_index:start]  # non-entity text before the current entity
        color = color_map[type]
        html_str += (   # entity text with the corresponding color
            f'<mark style="background-color: {color}">{labeled_text2.text[start:stop]}'
            + f'<sub>({type})</sub></mark>'
        )
        last_index = stop
    html_str += labeled_text2.text[last_index:]  # remaining text after the last entity
    display(HTML(html_str))

In [12]:
_text_idx = 7
print(test_data[_text_idx], '\n')
visualize_labeled_text(test_data[_text_idx], COLOR_MAP)

LabeledText(text='Барак Обама назначил губернатора Юты послом США в Китае\nПрезидент США Барак Обама 16 мая назначил губернатора штата Юта Джона Хантсмена-младшего (John Huntsman Jr.) послом США в Китае, пишет The Washington Times.\n\nПо словам представителя Белого дома, кандидатуру Хантсмена Обаме предложил помощник по азиатской политике Джефф Бэйдер (Jeff Bader), который представил губернатора Юты как человека, отлично знающего китайский язык, разбирающегося в проблемах региона и способного эффективно решать дипломатические задачи.\n\nЧлены Республиканской партии, в том числе и бывший соперник Обамы на выборах президента США Джон Маккейн, поприветствовали выбор Обамы. Они, однако, в то же время признали, что в связи с этим назначением Хантсмен фактически лишился возможности участвовать в следующих президентских выборах.\n\nВо время предвыборной кампании 2008 года Хантсмен был одним из руководителей штаба Джона Маккейна и, по оценкам экспертов, именно он мог стать кандидатом от респуб

In [13]:
_text_idx = 13

print(test_data[_text_idx], '\n')
visualize_labeled_text(test_data[_text_idx], COLOR_MAP)

LabeledText(text='Hermitage: Главу налогового управления ГУВД Москвы уволили из-за С.Магнитского\n\nФонд Hermitage Capital связывает освобождение генерал-майора милиции Анатолия Михалкина от должности начальника управления по налоговым преступлениям ГУВД Москвы с делом юриста фонда Сергея Магнитского, умершего 16 ноября 2009г. в следственном изоляторе, говорится в распространенном сегодня сообщении Hermitage Capital.\n\nПо информации фонда, уволенный генерал-майор ГУВД Москвы был руководителем подполковника А.Кузнецова, против которого С.Магнитский давал показания. Кроме того, в июле-августе 2007г. А.Михалкин направил ряд запросов, подготовленных А.Кузнецовым, в московские филиалы международных банков с целью получения конфиденциальной информации о российских компаниях фонда Hermitage, утверждается в сообщении компании.', entities=[Ne5Span(start=0, stop=9, type='ORG'), Ne5Span(start=39, stop=43, type='ORG'), Ne5Span(start=44, stop=50, type='LOC'), Ne5Span(start=65, stop=78, type='PER')

# 2. Tokenize dataset

In [ ]:
# Токенизация и выравнивание меток
def tokenize_rus(text: str) -> list[Substring]:
    """Implement word-wise tokenization for russian language."""
    return list(razdel.tokenize(text))

In [15]:
def symbol_bio_to_word_bio(
    labeled_text: LabeledText,
    subwords: list[Substring],
) -> list[str]:
    """Transforms symbol-wise annotation to word-wise BIO notation for NER.

    Args:
        labeled_text: A LabeledText object containing the original text and
            symbol-wise entity annotations.
        subwords: A list of Substring objects representing the tokenized text.

    Returns:
        A list of strings representing BIO tags in the format:
        - "B-CAT" for beginning of an entity of category CAT
        - "I-CAT" for continuation of an entity of category CAT
        - "O" for tokens outside any entity
    """
    bio_tags = ["O"] * len(subwords)
    entities = sorted(labeled_text.entities, key=lambda x: x.start)
    for entity in entities:
        overlapping = []
        for i, subword in enumerate(subwords):
            if (subword.start < entity.stop and subword.stop > entity.start):
                overlapping.append(i)
        for j, idx in enumerate(overlapping):
            bio_tags[idx] = f"B-{entity.type}" if j == 0 else f"I-{entity.type}"
    return bio_tags

In [16]:
def align_labels_with_tokens(labels: list[str], word_ids: list[str]) -> list[str]:
    """Aligns word-level BIO labels to token-level labels accounting for wordpiece tokenization.
    
    Args:
        labels: List of BIO worl-level labels.
        word_ids: List mapping each token to its original word index (from tokenizer.word_ids()).
    
    Returns:
        List of aligned token-level BIO labels.
    """
    new_labels = []
    crnt_word_id = None
    for word_id in word_ids:
        if word_id is None:
            new_labels.append('Ignored')  # special token
        elif word_id != crnt_word_id:
            crnt_word_id = word_id
            new_labels.append(labels[word_id])
        else:
            label = labels[word_id]
            if label.startswith("B-"):
                label = f"I-{label[2:]}"
            new_labels.append(label)
    return new_labels

In [17]:
class LabelsTokenizerAligner:
    def __init__(self, bio_labels_to_idx: dict[str, int], tokenizer):
        self.bio_labels_to_idx = bio_labels_to_idx
        self.tokenizer = tokenizer

    def _tokenize_and_align_labels(self, examples: Dataset):
        """Tokenizes input text and aligns word-level BIO labels with subword tokens.
    
        Args:
            examples (Dataset): A Hugging Face `Dataset` object containing:
                - `words`: List of words.
                - `labels_bio`: List of word-level BIO labels.
            tokenizer (PreTrainedTokenizer): Tokenizer (e.g., `BertTokenizer`) with subword tokenization.
    
        Returns:
            (Dataset): A Hugging Face `Dataset` object.
        """
        tokenized_inputs = self.tokenizer(
            examples["words"], 
            truncation=True, 
            padding=False,  # we will pad later with data_collator
            is_split_into_words=True
        )
        aligned_labels = []
        for sample_idx, bio_labels in enumerate(examples["bio_labels"]):
            word_ids = tokenized_inputs.word_ids(batch_index=sample_idx)
            bio_labels_aligned = align_labels_with_tokens(bio_labels, word_ids)
            aligned_labels.append([self.bio_labels_to_idx[bla] for bla in bio_labels_aligned])
        tokenized_inputs["labels"] = aligned_labels
        return tokenized_inputs

    def __call__(self, examples: Dataset):
        return self._tokenize_and_align_labels(examples)

In [18]:
def make_tokenized_dataset(
    ds_ner: list[LabeledText],
    bio_labels_to_idx: dict[str, int],
    tokenizer,
) -> Dataset:
    """Make tokenized dataset ready for batching.

    Makes(Dataset): a Hugging Face `Dataset` object containing model inputs:
        input_ids, token_type_ids, and an attention_mask via the steps:
        1) Represent raw dataset as python dictionary of words and bio_labels.
        2) Split sentences into words and convert symbol-wise labeling into word-wise BIO format.
        3) Tokenize words and align the labels with sub-tokens level.

    Args:
        ds_ner: List of (LabeledText) objects.
        bio_labels_to_idx: Map for BIO labels to int including 'Ignored' key for special tokens.
        tokenizer (PreTrainedTokenizer): Tokenizer (e.g., `BertTokenizer`) with subword tokenization.

    Returns:
        (Dataset): A tokenized Hugging Face `Dataset` object.
    """
    ds_dict = {"words": [], "bio_labels": []}
    for sample in ds_ner:
        words_subs = tokenize_rus(sample.text)
        words = [ws.text for ws in words_subs]
        bio_labels = symbol_bio_to_word_bio(sample, words_subs)
        ds_dict["words"].append(words)
        ds_dict["bio_labels"].append(bio_labels)
    dataset = Dataset.from_dict(ds_dict)
    labels_tokenizer_aligner = LabelsTokenizerAligner(bio_labels_to_idx, tokenizer)
    dataset_tokenized = dataset.map(             # The tokenizer generates three new columns in the
        labels_tokenizer_aligner,                # dataset: input_ids, token_type_ids, and an
        batched=True,                            # attention_mask. These are the model inputs.
        remove_columns=['words', 'bio_labels'],  # <-- We should delete the rest as we don't need them.
    )
    return dataset_tokenized

In [19]:
bio_labels_to_idx = {'O': 0}
for cat in categories_map:
    for prefix in 'BI':
        bio_labels_to_idx[f'{prefix}-{cat}'] = len(bio_labels_to_idx)
bio_labels_to_idx['Ignored'] = -100

idx_to_bio_labels = {lbl: bio for bio, lbl in bio_labels_to_idx.items()}

bio_labels_to_idx

{'O': 0,
 'B-GEOPOLIT': 1,
 'I-GEOPOLIT': 2,
 'B-LOC': 3,
 'I-LOC': 4,
 'B-MEDIA': 5,
 'I-MEDIA': 6,
 'B-PER': 7,
 'I-PER': 8,
 'B-ORG': 9,
 'I-ORG': 10,
 'Ignored': -100}

In [ ]:
# Подготавливаем tokenizer и датасеты
model_checkpoint = "cointegrated/rubert-tiny2"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [21]:
ds_tokenized_test = make_tokenized_dataset(test_data, bio_labels_to_idx, tokenizer)
ds_tokenized_train = make_tokenized_dataset(train_data, bio_labels_to_idx, tokenizer)

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

Map:   0%|          | 0/800 [00:00<?, ? examples/s]

In [22]:
print(f'test size: {len(ds_tokenized_test)}')
print(f'train size: {len(ds_tokenized_train)}')

test size: 200
train size: 800


In [23]:
_test_idx = 0
print(f'raw test: {test_data[_test_idx].text}')
_tokenized_decoded = tokenizer.decode(ds_tokenized_test[_test_idx]['input_ids'], skip_special_tokens=True)
print(f"tok test: {_tokenized_decoded}")

raw test: 4 октября назначены очередные выборы Верховного Совета Аджарской АР

По распоряжению президента Грузии Михаила Саакашвили 4 октября 2008 года назначены очередные выборы Верховного Совета Аджарской АР. Об этом Новости-Грузия сообщили в пресс-службе администрации президента во вторник.

Выборы Верховного совета Аджарской автономной республики назначены в соответствии с 241-ой статьей и 4-м пунктом 10-й статьи Конституционного закона Грузии <О статусе Аджарской автономной республики>.
tok test: 4 октября назначены очередные выборы Верховного Совета Аджарской АР По распоряжению президента Грузии Михаила Саакашвили 4 октября 2008 года назначены очередные выборы Верховного Совета Аджарской АР. Об этом Новости - Грузия сообщили в пресс - службе администрации президента во вторник. Выборы Верховного совета Аджарской автономной республики назначены в соответствии с 241 - ой статьей и 4 - м пунктом 10 - й статьи Конституционного закона Грузии < О статусе Аджарской автономной республики

# Дообучение модели rubert-tiny2

## 3.1 Training

In [24]:
data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

In [25]:
label2id = {label: idx for label, idx in bio_labels_to_idx.items() if idx >= 0}
id2label = {idx: label for label, idx in label2id.items()}

In [26]:
id2label

{0: 'O',
 1: 'B-GEOPOLIT',
 2: 'I-GEOPOLIT',
 3: 'B-LOC',
 4: 'I-LOC',
 5: 'B-MEDIA',
 6: 'I-MEDIA',
 7: 'B-PER',
 8: 'I-PER',
 9: 'B-ORG',
 10: 'I-ORG'}

In [27]:
model = AutoModelForTokenClassification.from_pretrained(
    model_checkpoint,
    num_labels=len(label2id),
    id2label=id2label,
    label2id=label2id
)

Some weights of BertForTokenClassification were not initialized from the model checkpoint at cointegrated/rubert-tiny2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [28]:
model.config.num_labels

11

In [29]:
metric = evaluate.load("seqeval")
label_names = list(label2id.keys()) 

def compute_metrics(eval_preds: tuple[np.ndarray, np.ndarray]) -> dict[str, float]:
    """
    Compute evaluation metrics for token classification (NER) using seqeval.

    Args:
        eval_preds (tuple): A tuple containing:
            - logits (np.ndarray): Model output logits of shape (batch_size, seq_len, num_labels)
            - labels (np.ndarray): Ground truth label ids of shape (batch_size, seq_len)

    Returns:
        dict: A dictionary with overall precision, recall, F1 score, and accuracy.
    """
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)

    true_labels = [[label_names[l] for l in label if l != -100] for label in labels]
    true_predictions = [
        [label_names[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    metrics = metric.compute(predictions=true_predictions, references=true_labels, zero_division=0)

    return {
        "precision": metrics["overall_precision"],
        "recall": metrics["overall_recall"],
        "f1": metrics["overall_f1"],
        "accuracy": metrics["overall_accuracy"],
    }

In [ ]:
# — оптимизатор AdamW (по умолчанию в HF Trainer)
# — lr=2e-5 — стандарт для дообучения BERT-подобных
# — batch_size=8 — укладывается в 8–12 GB VRAM
# — epochs=20 — даёт стабильную сходимость на малых корпусах

training_args = TrainingArguments(
    output_dir="./ner_model",
    eval_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=25,
    weight_decay=0.01,
    seed=random_state,
)

In [31]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=ds_tokenized_train,
    eval_dataset=ds_tokenized_test,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

In [32]:
print("Метрики до дообучения:")
trainer.evaluate(ds_tokenized_test)

Метрики до дообучения:


{'eval_loss': 2.4798085689544678,
 'eval_model_preparation_time': 0.0013,
 'eval_precision': 0.002396244574077348,
 'eval_recall': 0.02341201304931875,
 'eval_f1': 0.004347516214097356,
 'eval_accuracy': 0.04799117546530777,
 'eval_runtime': 1.7207,
 'eval_samples_per_second': 116.234,
 'eval_steps_per_second': 14.529}

In [33]:
trainer.train()

Epoch,Training Loss,Validation Loss,Model Preparation Time,Precision,Recall,F1,Accuracy
1,No log,0.880844,0.001300,0.101950,0.048167,0.065424,0.763291
2,No log,0.712447,0.001300,0.128584,0.085204,0.102493,0.794565
3,No log,0.620033,0.001300,0.171195,0.136634,0.151974,0.814343
4,No log,0.560742,0.001300,0.200128,0.180196,0.189640,0.827145
5,0.806500,0.516552,0.001300,0.235746,0.211860,0.223166,0.836762
6,0.806500,0.487550,0.001300,0.247217,0.247169,0.247193,0.843054
7,0.806500,0.460214,0.001300,0.264335,0.269814,0.267047,0.849330
8,0.806500,0.440893,0.001300,0.281449,0.287661,0.284521,0.854069
9,0.806500,0.424094,0.001300,0.296262,0.305699,0.300907,0.859522
10,0.459700,0.411086,0.001300,0.304723,0.318173,0.311303,0.862023


TrainOutput(global_step=2500, training_loss=0.45330238647460935, metrics={'train_runtime': 323.1975, 'train_samples_per_second': 61.882, 'train_steps_per_second': 7.735, 'total_flos': 177478250391600.0, 'train_loss': 0.45330238647460935, 'epoch': 25.0})

In [34]:
print("Метрики после дообучения:")
trainer.evaluate(ds_tokenized_test)

Метрики после дообучения:


{'eval_loss': 0.3687644302845001,
 'eval_model_preparation_time': 0.0013,
 'eval_precision': 0.3323353293413174,
 'eval_recall': 0.362118595279217,
 'eval_f1': 0.34658830011938657,
 'eval_accuracy': 0.8721996084889538,
 'eval_runtime': 1.3794,
 'eval_samples_per_second': 144.992,
 'eval_steps_per_second': 18.124,
 'epoch': 25.0}

## 3.2 Evaluation

In [35]:
predictions_output = trainer.predict(ds_tokenized_test)
logits = predictions_output.predictions
labels = predictions_output.label_ids
preds = np.argmax(logits, axis=-1)

true_labels = [[label_names[l] for l in label if l != -100] for label in labels]
pred_labels = [[label_names[p] for (p, l) in zip(pred, label) if l != -100] for pred, label in zip(preds, labels)]

y_true_flat = [l for seq in true_labels for l in seq]
y_pred_flat = [l for seq in pred_labels for l in seq]

cm = confusion_matrix(y_true_flat, y_pred_flat, labels=label_names)
cm_df = pd.DataFrame(cm, index=label_names, columns=label_names)
print("Confusion Matrix:")
cm_df

Confusion Matrix:


,O,B-GEOPOLIT,I-GEOPOLIT,B-LOC,I-LOC,B-MEDIA,I-MEDIA,B-PER,I-PER,B-ORG,I-ORG
O,43495,97,102,59,261,39,89,344,1052,266,713
B-GEOPOLIT,137,392,51,17,10,1,0,23,7,14,46
I-GEOPOLIT,147,58,225,3,41,0,0,6,41,1,61
B-LOC,81,22,5,344,124,0,0,26,10,13,12
I-LOC,197,4,28,22,1087,0,0,13,36,7,47
B-MEDIA,71,0,0,2,0,167,33,5,0,12,5
I-MEDIA,106,0,2,0,2,22,400,11,14,1,77
B-PER,292,4,2,8,6,0,3,1373,515,12,19
I-PER,669,2,6,2,21,3,2,243,5580,9,31
B-ORG,334,17,0,21,2,8,3,23,11,673,255


In [36]:
print("\nClassification Report:")
print(classification_report(true_labels, pred_labels))


Classification Report:
              precision    recall  f1-score   support

    GEOPOLIT       0.39      0.40      0.40       698
         LOC       0.37      0.38      0.37       637
       MEDIA       0.35      0.37      0.36       295
         ORG       0.25      0.28      0.26      1347
         PER       0.35      0.39      0.37      2234

   micro avg       0.33      0.36      0.35      5211
   macro avg       0.34      0.36      0.35      5211
weighted avg       0.33      0.36      0.35      5211



In [37]:
ner_pipeline = pipeline("ner", model=model, tokenizer=tokenizer, aggregation_strategy="simple", device=0)

text = "Барак Обама назначил губернатора Юты послом США в Китае Президент США Барак Обама 16 мая назначил губернатора штата Юта Джона Хантсмена-младшего (John Huntsman Jr.) послом США в Китае, пишет The Washington Times. По словам представителя Белого дома, кандидатуру Хантсмена Обаме предложил помощник по азиатской политике Джефф Бэйдер (Jeff Bader), который представил губернатора Юты как человека, отлично знающего китайский язык, разбирающегося в проблемах региона и способного эффективно решать дипломатические задачи. Члены Республиканской партии, в том числе и бывший соперник Обамы на выборах президента США Джон Маккейн, поприветствовали выбор Обамы. Они, однако, в то же время признали, что в связи с этим назначением Хантсмен фактически лишился возможности участвовать в следующих президентских выборах. Во время предвыборной кампании 2008 года Хантсмен был одним из руководителей штаба Джона Маккейна и, по оценкам экспертов, именно он мог стать кандидатом от республиканцев на следующих выборах. Многие республиканцы отметили, что, сделав Хантсмена послом, Обама устранил потенциально опасного соперника. До избрания губернатором Юты в 2004 году Хантсмен работал торговым представителем США при Джордже Буше-младшем, а до этого был послом США в Сингапуре."
entities = ner_pipeline(text)
print(entities)

predicted_entities = [
    Ne5Span(start=ent["start"], stop=ent["end"], type=ent["entity_group"])
    for ent in entities
]

predicted_labeled_text = LabeledText(text=text, entities=predicted_entities)

visualize_labeled_text(predicted_labeled_text, COLOR_MAP)

Device set to use cuda:0


[{'entity_group': 'PER', 'score': 0.9145874, 'word': 'Барак Обама', 'start': 0, 'end': 11}, {'entity_group': 'PER', 'score': 0.45619935, 'word': 'Ю', 'start': 33, 'end': 34}, {'entity_group': 'GEOPOLIT', 'score': 0.55067486, 'word': '##ты', 'start': 34, 'end': 36}, {'entity_group': 'GEOPOLIT', 'score': 0.8653656, 'word': 'США', 'start': 44, 'end': 47}, {'entity_group': 'GEOPOLIT', 'score': 0.8083028, 'word': 'Китае', 'start': 50, 'end': 55}, {'entity_group': 'GEOPOLIT', 'score': 0.85899687, 'word': 'США', 'start': 66, 'end': 69}, {'entity_group': 'PER', 'score': 0.84993076, 'word': 'Барак Обама 16', 'start': 70, 'end': 84}, {'entity_group': 'PER', 'score': 0.84370923, 'word': 'Ю', 'start': 116, 'end': 117}, {'entity_group': 'LOC', 'score': 0.29862353, 'word': '##та', 'start': 117, 'end': 119}, {'entity_group': 'PER', 'score': 0.93264985, 'word': 'Джона Хантсмена - младшего ( John Huntsman Jr. )', 'start': 120, 'end': 164}, {'entity_group': 'GEOPOLIT', 'score': 0.5199127, 'word': 'США в

In [38]:
trainer.save_model("./ner_finetuned_model")

In [39]:
del trainer
del tokenizer
del data_collator
gc.collect()
torch.cuda.empty_cache()

# 4. Дообучение в MLM режиме

## 4.1 Training

- цель: донастроить LM-часть модели на доменных текстах перед дообучением NER
- block_size=512 — максимальный контекст для Rubert-tiny2
- mlm_probability=0.15 — по умолчанию в BERT
- повторная fine-tuning тех же параметров lr/batch/epochs

In [40]:
block_size = 512

def group_texts(examples):
    # Concatenate all texts.
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the small remainder, we could add padding if the model supported it instead of this drop, you can
    # customize this part to your needs.
    if total_length >= block_size:
        total_length = (total_length // block_size) * block_size
    # Split by chunks of block_size.
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

In [41]:
def preprocess_function(examples):
    return tokenizer(examples["text"])

In [42]:
ds_tokenized_mlm = ds_tokenized_train.map(group_texts, batched=True)
ds_tokenized_mlm_test = ds_tokenized_test.map(group_texts, batched=True)

Map:   0%|          | 0/800 [00:00<?, ? examples/s]

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

In [43]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model = AutoModelForMaskedLM.from_pretrained(model_checkpoint)
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm_probability=0.15)

In [44]:
mlm_args  = TrainingArguments(
    output_dir="./mlm_model",
    eval_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=25,
    weight_decay=0.01,
    seed=random_state,
)

In [45]:
mlm_trainer = Trainer(
    model=model,
    args=mlm_args ,
    train_dataset=ds_tokenized_mlm,
    eval_dataset=ds_tokenized_mlm, #использую тоже трейн чтобы не было утечки
    data_collator=data_collator,
    processing_class=tokenizer,
)

In [46]:
mlm_trainer.evaluate(ds_tokenized_mlm_test)

{'eval_loss': 3.0680694580078125,
 'eval_model_preparation_time': 0.002,
 'eval_runtime': 1.0423,
 'eval_samples_per_second': 120.885,
 'eval_steps_per_second': 15.35}

In [47]:
mlm_trainer.train()

Epoch,Training Loss,Validation Loss,Model Preparation Time
1,No log,3.058262,0.002000
2,No log,3.013006,0.002000
3,No log,2.933980,0.002000
4,No log,2.888930,0.002000
5,No log,2.864507,0.002000
6,No log,2.883245,0.002000
7,No log,2.849713,0.002000
8,No log,2.804169,0.002000
9,3.225500,2.817735,0.002000
10,3.225500,2.763472,0.002000


TrainOutput(global_step=1525, training_loss=3.0757911857229763, metrics={'train_runtime': 333.2921, 'train_samples_per_second': 36.379, 'train_steps_per_second': 4.576, 'total_flos': 92534610432000.0, 'train_loss': 3.0757911857229763, 'epoch': 25.0})

In [48]:
mlm_trainer.evaluate()

{'eval_loss': 2.6637213230133057,
 'eval_model_preparation_time': 0.002,
 'eval_runtime': 3.3615,
 'eval_samples_per_second': 144.28,
 'eval_steps_per_second': 18.147,
 'epoch': 25.0}

In [49]:
mlm_trainer.save_model("./mlm_model")
del mlm_trainer
gc.collect()
torch.cuda.empty_cache()

In [ ]:
# Постобучение NER после MLM
model = AutoModelForTokenClassification.from_pretrained("./mlm_model", num_labels=len(id2label), id2label=id2label, label2id=label2id)
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

Some weights of BertForTokenClassification were not initialized from the model checkpoint at ./mlm_model and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [51]:
post_args  = TrainingArguments(
    output_dir="./post_mlm_model",
    eval_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=25,
    weight_decay=0.01,
    seed=random_state,
)

In [52]:
post_mlm_trainer = Trainer(
    model=model,
    args=post_args ,
    train_dataset=ds_tokenized_train,
    eval_dataset=ds_tokenized_test, 
    data_collator=data_collator,
)

In [53]:
print("Метрики до дообучения:")
post_mlm_trainer.evaluate(ds_tokenized_test)
compute_metrics(
    (post_mlm_trainer.predict(ds_tokenized_test).predictions, ds_tokenized_test['labels'])
     )

Метрики до дообучения:


{'precision': 0.0026155591046918038,
 'recall': 0.021684897332565727,
 'f1': 0.004668071219068865,
 'accuracy': 0.12991330826834044}

In [54]:
post_mlm_trainer.train()

Epoch,Training Loss,Validation Loss,Model Preparation Time
1,No log,0.864322,0.000500
2,No log,0.689883,0.000500
3,No log,0.601047,0.000500
4,No log,0.540374,0.000500
5,0.784600,0.495677,0.000500
6,0.784600,0.465287,0.000500
7,0.784600,0.439280,0.000500
8,0.784600,0.421655,0.000500
9,0.784600,0.408100,0.000500
10,0.440500,0.397149,0.000500


TrainOutput(global_step=2500, training_loss=0.4376122497558594, metrics={'train_runtime': 310.1322, 'train_samples_per_second': 64.489, 'train_steps_per_second': 8.061, 'total_flos': 177478250391600.0, 'train_loss': 0.4376122497558594, 'epoch': 25.0})

In [55]:
print("Метрики после дообучения:")
post_mlm_trainer.evaluate()
compute_metrics(
    (post_mlm_trainer.predict(ds_tokenized_test).predictions, ds_tokenized_test['labels'])
     )

Метрики после дообучения:


{'precision': 0.3410631466286122,
 'recall': 0.3669161389368643,
 'f1': 0.3535176111676065,
 'accuracy': 0.8747475375198086}

## 4.2 Evaluation

In [56]:
predictions_output = post_mlm_trainer.predict(ds_tokenized_test)
logits = predictions_output.predictions
labels = predictions_output.label_ids
preds = np.argmax(logits, axis=-1)

true_labels = [[label_names[l] for l in label if l != -100] for label in labels]
pred_labels = [[label_names[p] for (p, l) in zip(pred, label) if l != -100] for pred, label in zip(preds, labels)]

y_true_flat = [l for seq in true_labels for l in seq]
y_pred_flat = [l for seq in pred_labels for l in seq]

cm = confusion_matrix(y_true_flat, y_pred_flat, labels=label_names)
cm_df = pd.DataFrame(cm, index=label_names, columns=label_names)
print("Confusion Matrix:")
cm_df

Confusion Matrix:


,O,B-GEOPOLIT,I-GEOPOLIT,B-LOC,I-LOC,B-MEDIA,I-MEDIA,B-PER,I-PER,B-ORG,I-ORG
O,43683,105,105,48,237,32,106,286,983,240,692
B-GEOPOLIT,132,388,59,18,9,0,0,19,6,19,48
I-GEOPOLIT,152,57,230,3,40,0,0,5,35,1,60
B-LOC,96,15,4,347,116,0,1,21,11,11,15
I-LOC,202,3,25,20,1109,0,1,8,30,4,39
B-MEDIA,70,0,0,2,0,159,43,4,0,14,3
I-MEDIA,107,0,4,1,3,14,427,10,17,1,51
B-PER,313,9,3,11,14,1,5,1343,498,14,23
I-PER,687,5,7,1,20,3,3,246,5550,7,39
B-ORG,358,10,1,25,1,4,4,18,8,673,245


In [57]:
print("\nClassification Report:")
print(classification_report(true_labels, pred_labels))


Classification Report:
              precision    recall  f1-score   support

    GEOPOLIT       0.41      0.42      0.42       698
         LOC       0.37      0.39      0.38       637
       MEDIA       0.36      0.37      0.36       295
         ORG       0.26      0.28      0.27      1347
         PER       0.36      0.40      0.38      2234

   micro avg       0.34      0.37      0.35      5211
   macro avg       0.35      0.37      0.36      5211
weighted avg       0.34      0.37      0.35      5211



In [58]:
ner_pipeline = pipeline("ner", model=model, tokenizer=tokenizer, aggregation_strategy="simple", device=0)

text = "Барак Обама назначил губернатора Юты послом США в Китае Президент США Барак Обама 16 мая назначил губернатора штата Юта Джона Хантсмена-младшего (John Huntsman Jr.) послом США в Китае, пишет The Washington Times. По словам представителя Белого дома, кандидатуру Хантсмена Обаме предложил помощник по азиатской политике Джефф Бэйдер (Jeff Bader), который представил губернатора Юты как человека, отлично знающего китайский язык, разбирающегося в проблемах региона и способного эффективно решать дипломатические задачи. Члены Республиканской партии, в том числе и бывший соперник Обамы на выборах президента США Джон Маккейн, поприветствовали выбор Обамы. Они, однако, в то же время признали, что в связи с этим назначением Хантсмен фактически лишился возможности участвовать в следующих президентских выборах. Во время предвыборной кампании 2008 года Хантсмен был одним из руководителей штаба Джона Маккейна и, по оценкам экспертов, именно он мог стать кандидатом от республиканцев на следующих выборах. Многие республиканцы отметили, что, сделав Хантсмена послом, Обама устранил потенциально опасного соперника. До избрания губернатором Юты в 2004 году Хантсмен работал торговым представителем США при Джордже Буше-младшем, а до этого был послом США в Сингапуре."
entities = ner_pipeline(text)
print(entities)

predicted_entities = [
    Ne5Span(start=ent["start"], stop=ent["end"], type=ent["entity_group"])
    for ent in entities
]

predicted_labeled_text = LabeledText(text=text, entities=predicted_entities)

visualize_labeled_text(predicted_labeled_text, COLOR_MAP)

Device set to use cuda:0


[{'entity_group': 'PER', 'score': 0.9038508, 'word': 'Барак Обама', 'start': 0, 'end': 11}, {'entity_group': 'PER', 'score': 0.40524822, 'word': 'Ю', 'start': 33, 'end': 34}, {'entity_group': 'GEOPOLIT', 'score': 0.64348817, 'word': '##ты', 'start': 34, 'end': 36}, {'entity_group': 'GEOPOLIT', 'score': 0.89571035, 'word': 'США', 'start': 44, 'end': 47}, {'entity_group': 'GEOPOLIT', 'score': 0.86792535, 'word': 'Китае', 'start': 50, 'end': 55}, {'entity_group': 'GEOPOLIT', 'score': 0.86807376, 'word': 'США', 'start': 66, 'end': 69}, {'entity_group': 'PER', 'score': 0.82137775, 'word': 'Барак Обама 16', 'start': 70, 'end': 84}, {'entity_group': 'PER', 'score': 0.48506346, 'word': 'Ю', 'start': 116, 'end': 117}, {'entity_group': 'LOC', 'score': 0.32584092, 'word': '##та', 'start': 117, 'end': 119}, {'entity_group': 'PER', 'score': 0.9545311, 'word': 'Джона Хантсмена - младшего ( John Huntsman Jr. )', 'start': 120, 'end': 164}, {'entity_group': 'GEOPOLIT', 'score': 0.6635784, 'word': 'США'

In [ ]:
post_mlm_trainer.save_model("./post_mlm_model")
del post_mlm_trainer
del tokenizer
del data_collator
gc.collect()
torch.cuda.empty_cache()

# 5. Использование доп разметки

## 5.1 Training

In [63]:
lenta = pd.read_parquet('data/synthetic_annotations.parquet')
lenta = lenta.rename(columns={'text': 'words', 'annotation': 'bio_labels'})
lenta.head()

,words,bio_labels
0,"[Египетский, перевозчик, EgyptAir, сообщил, о,...","[O, O, B-ORG, O, O, O, O, O, O, O, O, O, O, O,..."
1,"[Глава, Красногорского, района, Московской, об...","[O, B-LOC, I-LOC, B-LOC, I-LOC, B-PER, I-PER, ..."
2,"[Депутат, Виталий, Милонов, внес, в, Госдуму, ...","[O, B-PER, I-PER, O, O, B-ORG, O, O, O, O, O, ..."
3,"[Верховный, суд, Индии, разрешил, женщинам, в,...","[B-ORG, I-ORG, B-LOC, O, O, O, O, O, O, O, O, ..."
4,"[Россиянам, не, стоит, бояться, роста, цен, на...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."


In [64]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
lenta_dataset = Dataset.from_pandas(lenta, preserve_index=False)
labels_tokenizer_aligner = LabelsTokenizerAligner(bio_labels_to_idx, tokenizer)
dataset_tokenized = lenta_dataset.map(labels_tokenizer_aligner, batched =True, remove_columns=['words', 'bio_labels'])

Map:   0%|          | 0/9713 [00:00<?, ? examples/s]

In [65]:
ds_tokenized_train, dataset_tokenized

(Dataset({
     features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
     num_rows: 800
 }),
 Dataset({
     features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
     num_rows: 9713
 }))

In [66]:
combined_ds = concatenate_datasets([ds_tokenized_train, dataset_tokenized])

print(combined_ds)

Dataset({
    features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
    num_rows: 10513
})


In [67]:
model = AutoModelForTokenClassification.from_pretrained(
    model_checkpoint,
    num_labels=len(label2id),
    id2label=id2label,
    label2id=label2id
)
data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

Some weights of BertForTokenClassification were not initialized from the model checkpoint at cointegrated/rubert-tiny2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [68]:
training_args = TrainingArguments(
    output_dir="./ner_model",
    eval_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=25,
    weight_decay=0.01,
    seed=random_state,
)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=combined_ds,
    eval_dataset=ds_tokenized_test,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)


In [69]:
print("Метрики до дообучения:")
trainer.evaluate(ds_tokenized_test)

Метрики до дообучения:


{'eval_loss': 2.386500358581543,
 'eval_model_preparation_time': 0.0,
 'eval_precision': 0.00340171330194354,
 'eval_recall': 0.03147188639416619,
 'eval_f1': 0.006139792594811126,
 'eval_accuracy': 0.08927073299568095,
 'eval_runtime': 1.7142,
 'eval_samples_per_second': 116.674,
 'eval_steps_per_second': 14.584}

In [70]:
trainer.train()

Epoch,Training Loss,Validation Loss,Model Preparation Time,Precision,Recall,F1,Accuracy
1,0.204200,0.894058,0.000000,0.101039,0.123201,0.111025,0.776699
2,0.133100,0.664287,0.000000,0.156402,0.166187,0.161146,0.805736
3,0.110300,0.607886,0.000000,0.174483,0.181347,0.177849,0.816906
4,0.095000,0.540663,0.000000,0.198361,0.208981,0.203532,0.829475
5,0.085100,0.564435,0.000000,0.212177,0.222030,0.216992,0.832551
6,0.072600,0.539629,0.000000,0.223066,0.236807,0.229731,0.837834
7,0.061900,0.526838,0.000000,0.243942,0.262713,0.252980,0.843629
8,0.055900,0.542800,0.000000,0.255297,0.277490,0.265931,0.844949
9,0.053600,0.569877,0.000000,0.246544,0.270390,0.257917,0.843753
10,0.045800,0.526747,0.000000,0.274383,0.292074,0.282952,0.853385


TrainOutput(global_step=32875, training_loss=0.05902036816267006, metrics={'train_runtime': 2216.6895, 'train_samples_per_second': 118.566, 'train_steps_per_second': 14.831, 'total_flos': 1439291200762776.0, 'train_loss': 0.05902036816267006, 'epoch': 25.0})

In [71]:
print("Метрики после дообучения:")
trainer.evaluate(ds_tokenized_test)

Метрики после дообучения:


{'eval_loss': 0.6349139213562012,
 'eval_model_preparation_time': 0.0,
 'eval_precision': 0.30184411969380653,
 'eval_recall': 0.33294952984072157,
 'eval_f1': 0.31663472944611737,
 'eval_accuracy': 0.8579840288350993,
 'eval_runtime': 4.9496,
 'eval_samples_per_second': 40.408,
 'eval_steps_per_second': 5.051,
 'epoch': 25.0}

## 5.1 Evaluation

In [72]:
predictions_output = trainer.predict(ds_tokenized_test)
logits = predictions_output.predictions
labels = predictions_output.label_ids
preds = np.argmax(logits, axis=-1)

true_labels = [[label_names[l] for l in label if l != -100] for label in labels]
pred_labels = [[label_names[p] for (p, l) in zip(pred, label) if l != -100] for pred, label in zip(preds, labels)]

y_true_flat = [l for seq in true_labels for l in seq]
y_pred_flat = [l for seq in pred_labels for l in seq]

cm = confusion_matrix(y_true_flat, y_pred_flat, labels=label_names)
cm_df = pd.DataFrame(cm, index=label_names, columns=label_names)
print("Confusion Matrix:")
cm_df

Confusion Matrix:


,O,B-GEOPOLIT,I-GEOPOLIT,B-LOC,I-LOC,B-MEDIA,I-MEDIA,B-PER,I-PER,B-ORG,I-ORG
O,43487,80,67,198,208,26,70,528,827,424,602
B-GEOPOLIT,177,267,54,118,23,0,0,21,9,7,22
I-GEOPOLIT,242,17,169,3,85,0,0,11,39,7,10
B-LOC,66,15,3,423,95,0,0,15,9,4,7
I-LOC,251,2,20,42,1056,0,1,10,34,5,20
B-MEDIA,54,1,0,0,0,103,31,4,0,88,14
I-MEDIA,170,0,3,0,0,6,238,11,16,30,161
B-PER,304,6,0,3,6,0,0,1349,547,6,13
I-PER,1322,2,3,3,13,3,1,165,5031,10,15
B-ORG,243,10,0,17,3,2,0,18,8,774,272


In [73]:
print("\nClassification Report:")
print(classification_report(true_labels, pred_labels))


Classification Report:
              precision    recall  f1-score   support

    GEOPOLIT       0.42      0.29      0.34       698
         LOC       0.28      0.41      0.33       637
       MEDIA       0.34      0.24      0.28       295
         ORG       0.24      0.31      0.27      1347
         PER       0.32      0.35      0.34      2234

   micro avg       0.30      0.33      0.32      5211
   macro avg       0.32      0.32      0.31      5211
weighted avg       0.31      0.33      0.32      5211



In [74]:
ner_pipeline = pipeline("ner", model=model, tokenizer=tokenizer, aggregation_strategy="simple", device=0)

text = "Барак Обама назначил губернатора Юты послом США в Китае Президент США Барак Обама 16 мая назначил губернатора штата Юта Джона Хантсмена-младшего (John Huntsman Jr.) послом США в Китае, пишет The Washington Times. По словам представителя Белого дома, кандидатуру Хантсмена Обаме предложил помощник по азиатской политике Джефф Бэйдер (Jeff Bader), который представил губернатора Юты как человека, отлично знающего китайский язык, разбирающегося в проблемах региона и способного эффективно решать дипломатические задачи. Члены Республиканской партии, в том числе и бывший соперник Обамы на выборах президента США Джон Маккейн, поприветствовали выбор Обамы. Они, однако, в то же время признали, что в связи с этим назначением Хантсмен фактически лишился возможности участвовать в следующих президентских выборах. Во время предвыборной кампании 2008 года Хантсмен был одним из руководителей штаба Джона Маккейна и, по оценкам экспертов, именно он мог стать кандидатом от республиканцев на следующих выборах. Многие республиканцы отметили, что, сделав Хантсмена послом, Обама устранил потенциально опасного соперника. До избрания губернатором Юты в 2004 году Хантсмен работал торговым представителем США при Джордже Буше-младшем, а до этого был послом США в Сингапуре."
entities = ner_pipeline(text)
print(entities)

predicted_entities = [
    Ne5Span(start=ent["start"], stop=ent["end"], type=ent["entity_group"])
    for ent in entities
]

predicted_labeled_text = LabeledText(text=text, entities=predicted_entities)

visualize_labeled_text(predicted_labeled_text, COLOR_MAP)

Device set to use cuda:0


[{'entity_group': 'PER', 'score': 0.9976202, 'word': 'Барак Обама', 'start': 0, 'end': 11}, {'entity_group': 'PER', 'score': 0.9597403, 'word': 'Юты', 'start': 33, 'end': 36}, {'entity_group': 'GEOPOLIT', 'score': 0.9866236, 'word': 'США', 'start': 44, 'end': 47}, {'entity_group': 'GEOPOLIT', 'score': 0.9596627, 'word': 'Китае', 'start': 50, 'end': 55}, {'entity_group': 'GEOPOLIT', 'score': 0.959941, 'word': 'США', 'start': 66, 'end': 69}, {'entity_group': 'PER', 'score': 0.97175795, 'word': 'Барак Обама 16', 'start': 70, 'end': 84}, {'entity_group': 'LOC', 'score': 0.82748, 'word': 'Юта', 'start': 116, 'end': 119}, {'entity_group': 'PER', 'score': 0.9641198, 'word': 'Джона Хантсмена - младшего ( John Huntsman Jr. )', 'start': 120, 'end': 164}, {'entity_group': 'GEOPOLIT', 'score': 0.8426329, 'word': 'США в', 'start': 172, 'end': 177}, {'entity_group': 'GEOPOLIT', 'score': 0.929965, 'word': 'Китае,', 'start': 178, 'end': 184}, {'entity_group': 'MEDIA', 'score': 0.62432885, 'word': 'The

In [75]:
trainer.save_model("./ner_finetuned_model")

- MLM-предтренировка перед NER дала наилучшие метрики: F1 (0.3535) и чуть более высокую точность и полноту по сравнению с прямым fine‑tuning без MLM (0.3466) и с fine‑tuning с синтетикой.
- Добавление синтетических аннотаций ухудшило все ключевые метрики (F1 упало до 0.3166), вероятно из‑за низкого качества или несоответствия синтетики реальному распределению. В особенности из-за того, что в синтетике не присутствовали теги GEOPOLIt и MEDIA, из-за чего показатели упали для типов сущностей.

| Approach        | Precision  | Recall     | F1         | Accuracy   |
| --------------- | ----------:| ----------:| ----------:| ----------:|
| Baseline FT     | 0.33233533 | 0.36211860 | 0.34658830 | 0.87219961 |
| MLM FT          | 0.34106315 | 0.36691614 | 0.35351761 | 0.87474754 |
| With Synthetic  | 0.30184412 | 0.33294953 | 0.31663473 | 0.85798403 |
